ใน ep นี้

# 0. Install

เราจะต้อง Install kaggle เพื่อ Download Dataset, geopandas เพื่อใช้ในการวิเคราะห์ข้อมูล geospatial, folium เพื่อแสดงแผ่นที่ (ถ้ายังไม่ได้ Install ให้ uncomment)

In [0]:
# !apt install gdal-bin python-gdal python3-gdal 
# # Install rtree - Geopandas requirment
# !apt install python3-rtree 
# # Install Geopandas
# !pip install git+git://github.com/geopandas/geopandas.git
# # Install descartes - Geopandas requirment
# !pip install descartes 

In [0]:
# ! pip install git+https://github.com/python-visualization/folium
# ! pip install kaggle --upgrade

# 1. Import Library

Import folium Library เพื่อใช้ในการพล็อตแผนที่แบบ Interactive

In [0]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode

import folium
from folium import *
from folium.plugins import *

import os
from pathlib import Path

ประกาศฟังก์ชันในการแสดงแผนที่โดยใช้ HTML iframe แต่ Colab ไม่ Support iframe เราจะ return Map ออกไปเลย

In [0]:
from IPython.display import IFrame, HTML

def embed_map(m, file_name):    
    m.save(file_name)
    # # VM
    # return IFrame(src=file_name, width='100%', height='500px')

    # Colab
    return m

# 2. เตรียม Path สำหรับดาวน์โหลดข้อมูล

กำหนด path ของ Config File และ Dataset ว่าจะอยู่ใน Google Drive ถ้าเราใช้ Google Colab หรือ อยู่ใน HOME ถ้าเราใช้ VM ธรรมดา และกำหนด Environment Variable ไปยังโฟลเดอร์ที่เก็บ kaggle.json

ในกรณีใช้ Colab ให้ Mount Google Drive เพื่อดึง Config File มาจาก Google Drive ส่วนตัวของเรา เมื่อเรารัน Cell ด้านล่างจะมีลิงค์ปรากฎขึ้นมาให้เรา Login กด Approve แล้ว Copy Authorization Code มาใส่ในช่องด้านล่าง แล้วกด Enter

In [0]:
dataset = 'alexisbcook/geospatial-learn-course-data'

# Google Colab
config_path = Path('/content/drive')
data_path = Path('/content/datasets/')/dataset
from google.colab import drive
drive.mount(str(config_path))
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

## VM
# config_path = Path(os.getenv("HOME"))
# data_path = config_path/"datasets"/dataset
# data_path.mkdir(parents=True, exist_ok=True)
# os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/.kaggle"

# 3. Dataset

ในเคสนี้ เราจะสมมติตัวเองเป็นตำรวจในเมืองบอสตัน เราจะใช้ Dataset ที่เรามีมาวิเคราะห์อาชญากรรม ที่เกิดขึ้นในเมือง Boston ป้องกันปราบปรามอาชญากรรม ในเมืองนี้ให้ลดน้อยลง

สั่งดาวน์โหลด Dataset จาก Kaggle พร้อมทั้ง unzip ไว้ใน data_path

In [0]:
!kaggle datasets download {dataset} -p "{data_path}" --unzip

# 4. Geocoding

In [7]:
result = geocode("Eiffel Tower", provider="nominatim", user_agent='BUA Labs/0.0.1')
result

,geometry,address
0,POINT (2.29450 48.85826),"Tour Eiffel, 5, Avenue Anatole France, Quartie..."


In [8]:
point = result.iloc[0].geometry
print(f'Latitude: {point.y}')
print(f'Longitude: {point.x}')
print(f'Address: {result.iloc[0].address}')

Latitude: 48.8582602
Longitude: 2.29449905431968
Address: Tour Eiffel, 5, Avenue Anatole France, Quartier du Gros-Caillou, Paris 7e Arrondissement, Paris, Île-de-France, France métropolitaine, 75007, France


In [9]:
universities = pd.read_csv(data_path/"top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [10]:
len(universities)

100

In [0]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim', user_agent='BUA Labs/0.0.1').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None


In [12]:
universities[['Latitude', 'Longitude', 'geometry']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

89.0% of addresses were geocoded!


In [13]:
# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.crs = {'init': 'epsg:4326'}
universities.head()

,Name,Latitude,Longitude,geometry
0,University of Oxford,51.753454,-1.254010,POINT (-1.25401 51.75345)
1,University of Cambridge,52.176390,0.143089,POINT (0.14309 52.17639)
2,Imperial College London,51.498871,-0.175608,POINT (-0.17561 51.49887)
3,ETH Zurich,47.376453,8.547709,POINT (8.54771 47.37645)
4,UCL,51.521682,-0.135208,POINT (-0.13521 51.52168)


# 5. Plot Map

In [14]:
# Create a map
m_1 = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m_1)

# Display the map
embed_map(m_1, '25e-m_1.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25e-m_1.html" frameborder="0" allowfullscreen></iframe>

# 6. Table Join

In [0]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [16]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


In [17]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


In [18]:
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ...",67106161,2699000.0
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300...",9960487,498100.0
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372...",9549747,165400.0


# 7. Spatial Join

In [0]:
# universities = universities[universities.is_valid]

In [20]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)
european_universities.head()

,Name,Latitude,Longitude,geometry,index_right,name,pop_est,gdp_md_est
0,University of Oxford,51.753454,-1.254010,POINT (-1.25401 51.75345),28,United Kingdom,64769452,2788000.0
1,University of Cambridge,52.176390,0.143089,POINT (0.14309 52.17639),28,United Kingdom,64769452,2788000.0
2,Imperial College London,51.498871,-0.175608,POINT (-0.17561 51.49887),28,United Kingdom,64769452,2788000.0
4,UCL,51.521682,-0.135208,POINT (-0.13521 51.52168),28,United Kingdom,64769452,2788000.0
5,London School of Economics and Political Science,51.514429,-0.116588,POINT (-0.11659 51.51443),28,United Kingdom,64769452,2788000.0


In [21]:
# Investigate the result
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))


We located 89 universities.
Only 83 of the universities were located in Europe (in 15 different countries).


aa


# Credit

* https://www.kaggle.com/alexisbcook/manipulating-geospatial-data
* http://geopandas.org/geocoding.html
* http://geopandas.org/mergingdata.html
* https://www.bualabs.com/archives/2744/geospatial-analysis-crimes-in-boston-usa-plot-interactive-map-crime-prevention-law-enforcement-folium-geospatial-ep-4/